In [31]:
!pip install pymorphy3

In [37]:
import re
import pymorphy3
import string

import kagglehub
import pandas as pd
from kagglehub import KaggleDatasetAdapter

path = kagglehub.dataset_download("yutkin/corpus-of-russian-news-articles-from-lenta")
file_path = "/root/.cache/kagglehub/datasets/yutkin/corpus-of-russian-news-articles-from-lenta/versions/2/lenta-ru-news.csv"

Using Colab cache for faster access to the 'corpus-of-russian-news-articles-from-lenta' dataset.


In [38]:
df = pd.read_csv(file_path)
df = df[['text']]

/tmp/ipython-input-720081923.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [39]:
df.head(10)

,text
0,Бои у Сопоцкина и Друскеник закончились отступ...
1,"Министерство народного просвещения, в виду про..."
2,"Штабс-капитан П. Н. Нестеров на днях, увидев в..."
3,Фотограф-корреспондент Daily Mirror рассказыва...
4,"Лица, приехавшие в Варшаву из Люблина, передаю..."
5,Как стало известно агентству Ассошиэйтед Пресс...
6,В зале игровых автоматов в третьем ярусе подзе...
7,Япония приняла решение разморозить кредиты Рос...
8,Британцы отмечают сегодня скорбную дату - втор...
9,В понедельник директор департамента внешних св...


In [45]:
sw = list(set([
    'типа', 'как бы', 'вот', 'это', 'это самое', 'так сказать', 'ну', 'значит', 'в общем', 'в общем-то',
    'вообще', 'вообще-то', 'короче', 'буквально', 'практически', 'скажем', 'конкретно', 'прикинь',
    'понимаешь', 'собственно', 'собственно говоря', 'фактически', 'статус', 'кстати', 'слушай', 'таки',
    'просто', 'прямо', 'реально', 'оказывается', 'следует отметить', 'не секрет', 'как известно',
    'известно', 'разумеется', 'безусловно', 'в настоящее время', 'на сегодняшний день', 'в данный момент',
    'имеет место быть', 'в силу того что', 'в связи с тем что', 'на самом деле', 'в принципе', 'в целом',
    'в частном случае', 'что касается', 'относительно', 'в связи с вышеизложенным', 'в результате',
    'в процессе', 'в рамках', 'в контексте', 'в случае', 'с точки зрения', 'исходя из', 'на основании',
    'в рамках реализации', 'в целях', 'для того чтобы', 'некоторый', 'определённый', 'кое-какой',
    'какой-то', 'что-то', 'где-то', 'как-то', 'почти', 'довольно', 'весьма', 'отчасти', 'в некоторой степени',
    'вполне', 'абсолютно', 'совершенно', 'очень', 'сильно', 'очень даже', 'нечто', 'кое-что', 'кое-где',
    'кое-как', 'является', 'являющийся', 'представляет собой', 'осуществлять', 'производить',
    'иметь место', 'выражаясь', 'говорят', 'говорит', 'говорил', 'говорилось', 'говорите', 'говорится',
    'говорить', 'говоря', 'выйти', 'делать', 'происходить', 'казаться', 'считаться', 'получается',
    'оказаться', 'присутствовать', 'отсутствовать', 'давать', 'получать', 'всё', 'это', 'то', 'вот это',
    'такой', 'такие', 'всё это', 'что', 'как', 'там', 'тут', 'здесь', 'ведь', 'же', 'вот', 'вон', 'да',
    'нет', 'ладно', 'хорошо', 'ух', 'эх', 'ой', 'ай', 'ого', 'ого-го', 'пиздец', 'блин', 'ёшкин кот',
    'ёпрст', 'чёрт', 'бля', 'офигенно', 'офигеть', 'обалдеть', 'жесть', 'ничего себе', 'вау', 'ого',
    'супер', 'круто', 'зашибись', 'прям', 'реально', 'и', 'а', 'но', 'да и', 'так вот', 'вот и',
    'вот так вот', 'ну и', 'а вообще', 'а в общем', 'а значит', 'а потом', 'а также', 'а ещё', 'при этом',
    'тем не менее', 'однако же'
]))

In [47]:
def calculate_stop_words(text):
  text_lis = text.lower().split()
  text_lis = list( map( lambda x: x.strip( string.punctuation ), text_lis ) )
  sw_in_text_lis = list( filter( lambda x: x in sw, text_lis ) )
  return f'Процент воды в тексте: { round( len( sw_in_text_lis ) / len(text_lis)*100, 1 ) }%'

In [33]:
morph = pymorphy3.MorphAnalyzer()

def count_syllables(word):
    vowels = 'аеёиоуыэюяАЕЁИОУЫЭЮЯ'
    return sum(1 for char in word if char in vowels)

def analyze_text_simple(text):
    sentences = [s.strip() for s in re.split(r'[.!?…]+', text) if s.strip()]

    words = re.findall(r'\b[а-яА-ЯёЁ]+\b', text)

    syllables = 0
    for word in words:
        parsed = morph.parse(word)[0]
        syllables += count_syllables(parsed.normal_form)

    return len(sentences), len(words), syllables

def calculate_readability_index(text):
  sentences, words, syllables = analyze_text_simple(text)

  if sentences > 0 and words > 0 and syllables > 0:
    index = 206.835 - 1.3 * (words/sentences) - 60.1 * (syllables/words)
  else:
    index = -1

  if index > 90:
    return index, "ОЧЕНЬ ВЫСОКИЙ"
  elif index > 80:
    return index, "ВЫСОКИЙ"
  elif index > 70:
    return index, "ВЫШЕ СРЕДНЕГО"
  elif index > 60:
    return index, "СРЕДНИЙ"
  elif index > 50:
    return index, "НИЖЕ СРЕДНЕГО"
  elif index > 30:
    return index, "НИЗКИЙ"
  elif index >= 0:
    return index, "ОЧЕНЬ НИЗКИЙ"
  else:
    return "ОШИБКА"

In [48]:
if __name__ == "__main__":
  for i in range(10):
    text = df['text'][i]
    print('=-'*20)
    print(text)
    print(calculate_readability_index(text))
    print(calculate_stop_words(text))

=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
Бои у Сопоцкина и Друскеник закончились отступлением германцев. Неприятель, приблизившись с севера к Осовцу начал артиллерийскую борьбу с крепостью. В артиллерийском бою принимают участие тяжелые калибры. С раннего утра 14 сентября огонь достиг значительного напряжения. Попытка германской пехоты пробиться ближе к крепости отражена. В Галиции мы заняли Дембицу. Большая колонна, отступавшая по шоссе от Перемышля к Саноку, обстреливалась с высот нашей батареей и бежала, бросив парки, обоз и автомобили. Вылазки гарнизона Перемышля остаются безуспешными. При продолжающемся отступлении австрийцев обнаруживается полное перемешивание их частей, захватываются новые партии пленных, орудия и прочая материальная часть. На перевале Ужок мы разбили неприятельский отряд, взяли его артиллерию и много пленных и, продолжая преследовать, вступили в пределы Венгрии. 
«Русский инвалид», 16 сентября 1914 года.
(46.39104743083004, 'НИЗКИЙ')
Процент воды в тексте: 5.1%

In [49]:
text = ' '

if __name__ == "__main__":
  while text != 'stop':
    text = input('Введите текст:')
    print('=-'*20)
    print(calculate_readability_index(text))
    print(calculate_stop_words(text))

Введите текст:В нашем динамично развивающемся городе, в условиях современной урбанистической среды, где так важен баланс между застройкой и зелёными зонами, произошло долгожданное и радостное событие. Сегодня, в свете реализации программы по благоустройству общественных пространств, можно с уверенностью констатировать факт официального открытия нового сквера. Нельзя не отметить, что данное мероприятие стало возможным благодаря слаженной работе всех причастных структур и органов власти, которые приложили немало усилий для того, чтобы горожане получили новое место для отдыха и релаксации.
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
(21.269684684684705, 'ОЧЕНЬ НИЗКИЙ')
Процент воды в тексте: 6.8%
Введите текст:На повестке дня очередного планового заседания экспертного комитета при городской администрации стоял целый ряд важных и актуальных вопросов, требующих детального рассмотрения и всестороннего обсуждения. Участники диалога, проявив высокий уровень профессионализма и компетентности, обме